In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
import time
from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.utils_chain import nominated_amount, dec_to_padded_hex
from utils.utils_scenarios import get_token_in_account
from contracts.metastaking_contract import MetaStakingContract, MetaStakingContractVersion
from events.metastake_events import EnterMetastakeEvent, ExitMetastakeEvent, ClaimRewardsMetastakeEvent
from utils.utils_generic import get_logger
from utils.utils_tx import ESDTToken

logger = get_logger("manual_interactor")

context = Context()

User & contract setup

In [ ]:
metastake_contract: MetaStakingContract
metastake_contract = context.get_contracts(config.METASTAKINGS_V2)[1]
print(metastake_contract.__dict__)

Collect users

In [ ]:
from utils.utils_scenarios import collect_farm_contract_users
from multiversx_sdk_network_providers import ApiNetworkProvider
from utils.utils_scenarios import FetchedUser
from typing import List

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
fetched_users = collect_farm_contract_users(10, metastake_contract.address, metastake_contract.farm_token, metastake_contract.metastake_token,
                                            mainnet_api, context.network_provider.proxy)

users: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
if not users:
    raise Exception('No users found for the given criteria')

In [ ]:
from utils.decoding_structures import METASTAKE_TOKEN_ATTRIBUTES, FARM_TOKEN_ATTRIBUTES, STAKE_V2_TOKEN_ATTRIBUTES, STAKE_V1_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, get_all_token_nonces_details_for_account, Account

index = 0
user = Account(pem_file="~/Documents/sh1.pem")
user.address = users[index].address
user.sync_nonce(context.network_provider.proxy)
tokens_in_account = get_all_token_nonces_details_for_account(metastake_contract.farm_token, user.address.bech32(), context.network_provider.proxy)
print(f'Account: {user.address.bech32()}')
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(metastake_contract.metastake_token, user.address.bech32(), context.network_provider.proxy)

print(f'Metastake Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    _, _, _ = metastake_contract.get_all_decoded_metastake_token_attributes_from_proxy(context.network_provider.proxy, user.address.bech32(), token["nonce"])


State

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from contracts.staking_contract import StakingContract
from contracts.farm_contract import FarmContract

def get_all_stats(metastake_contract: MetaStakingContract, user: Account):
    energy_contract: SimpleLockEnergyContract
    energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
    user_energy = energy_contract.get_energy_for_user(context.network_provider.proxy, user.address.bech32())
    logger.debug(f'User energy on energy factory: {user_energy}')

    staking_contract: StakingContract
    for contract in context.get_contracts(config.STAKINGS_V2):
        if contract.address == metastake_contract.stake_address:
            logger.debug(f"Found {contract.address}")
            staking_contract = contract
            break

    logger.debug(f"Stats for user: {user.address.bech32()} on staking {staking_contract.address}")
    staking_stats = staking_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
    staking_stats.update(staking_contract.get_all_stats(context.network_provider.proxy))
    logger.debug(f"Staking stats: {staking_stats}")
    
    farm_contract: FarmContract
    for contract in context.get_contracts(config.FARMS_V2):
        if contract.address == metastake_contract.farm_address:
            logger.debug(f"Found {contract.address}")
            farm_contract = contract
            break

    logger.debug(f"Stats for user: {user.address.bech32()} on farm {farm_contract.address}")
    farm_stats = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
    farm_stats.update(farm_contract.get_all_stats(context.network_provider.proxy))
    logger.debug(f"Farm stats: {farm_stats}")

    return user_energy, staking_stats, farm_stats

get_all_stats(metastake_contract, user)

In [ ]:
from utils.utils_chain import dec_to_padded_hex

def get_tx_op(tx_hash: str, operation: dict) -> dict:
    used_api = context.network_provider.api

    # Get the transaction details
    tx = used_api.get_transaction(tx_hash)

    # Get and check transaction operations
    ops = tx.raw_response['operations']

    # Take each op in ops and match it with operation. Try to match only the fields expected in operation dictionary. 
    # TX Operations are unordered. If any of the operations match, return it.
    for op in ops:
        # print(f'Matching with {operation}')
        if all(op.get(key) == operation.get(key) for key in operation.keys()):
            return op

upgrade all contracts

In [ ]:
# upgrade all contracts
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[MetaStakingContract] = context.get_contracts(config.METASTAKINGS_V2)
for contract in contracts:
    contract.version = MetaStakingContractVersion.V3Boosted
    tx_hash = contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         config.STAKING_PROXY_V3_BYTECODE_PATH, 
                                         [], no_init=True)
    context.network_provider.check_complex_tx_status(tx_hash, "proxy staking upgrade")

In [ ]:
# set energy factory address
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[MetaStakingContract] = context.get_contracts(config.METASTAKINGS_V2)
for contract in contracts:
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)

Enter metastake

In [ ]:
amount_percentage = 100

token_nonce, token_balance, token_attributes = get_token_in_account(context.network_provider.proxy, user, metastake_contract.farm_token)
if token_balance == 0:
    print("No token balance")
    exit(1)
token_balance_to_use = token_balance * amount_percentage // 100

user_energy_before, staking_stats_before, farm_stats_before = get_all_stats(metastake_contract, user)

tokens = [
    ESDTToken(metastake_contract.farm_token, token_nonce, token_balance_to_use)
]

tx_hash = metastake_contract.enter_metastake(context.network_provider, user, [tokens])

time.sleep(6 if user.address.get_shard() == 1 else 40)

user_energy_after, staking_stats_after, farm_stats_after = get_all_stats(metastake_contract, user)

Claim metastake

In [ ]:
amount_percentage = 100

token_nonce, token_balance, token_attributes = get_token_in_account(context.network_provider.proxy, user, metastake_contract.metastake_token)
if token_balance == 0:
    print("No token balance")
    raise
token_balance_to_use = token_balance * amount_percentage // 100

user_energy_before, staking_stats_before, farm_stats_before = get_all_stats(metastake_contract, user)

tokens = [
    ESDTToken(metastake_contract.metastake_token, token_nonce, token_balance_to_use)
]

tx_hash = metastake_contract.claim_rewards_metastaking(context.network_provider, user, [tokens])

time.sleep(6 if user.address.get_shard() == 1 else 40)

user_energy_after, staking_stats_after, farm_stats_after = get_all_stats(metastake_contract, user)

Check claim effects

In [ ]:
op_to_look_for = { # the Staked token transfered from staking contract to metastake contract
        "action": "transfer",
        "sender": metastake_contract.stake_address,
        "receiver": metastake_contract.address,
        "collection": metastake_contract.stake_token
    }
op = get_tx_op(tx_hash, op_to_look_for)
new_stake_token = op.get('identifier')
new_stake_token_value = int(op.get('value'))

op_to_look_for = { # the metataked token transfered from proxy staking contract to user
        "action": "transfer",
        "sender": metastake_contract.address,
        "receiver": user.address.bech32(),
        "collection": metastake_contract.metastake_token
    }
op = get_tx_op(tx_hash, op_to_look_for)
new_metastake_token = op.get('identifier')
new_metastake_token_value = int(op.get('value'))

# check the user_total_farm_position is increased with new safe price stake value
if staking_stats_before.get("user_total_farm_position") != {}:
    user_total_farm_position_base = staking_stats_before.get("user_total_farm_position").get("total_farm_position") - token_balance_to_use
else:
    user_total_farm_position_base = 0

user_total_farm_position = staking_stats_after.get("user_total_farm_position").get("total_farm_position")
if user_total_farm_position != user_total_farm_position_base + new_stake_token_value:
    print(f"Expected user_total_farm_position {token_balance_to_use} but got {user_total_farm_position}")

new_proxy_attrs, new_farm_attrs, new_stake_attrs = metastake_contract.get_all_decoded_metastake_token_attributes_from_proxy(context.network_provider.proxy, user.address.bech32(), 
                                                                                        int(new_metastake_token.split("-")[2], 16))
if new_farm_attrs.get("original_owner") != user.address.bech32():
    print(f"Expected original_owner {user.address.bech32()} but got {new_farm_attrs.get('original_owner')}")
else:
    print(f"Original owner is correct: {new_farm_attrs.get('original_owner')}")

if new_stake_attrs.get("original_owner") != user.address.bech32():
    print(f"Expected original_owner {user.address.bech32()} but got {new_stake_attrs.get('original_owner')}")
else:
    print(f"Original owner is correct: {new_stake_attrs.get('original_owner')}")

if user_energy_after.get("amount") != staking_stats_after.get("user_energy_for_week").get("amount"):
    print(f"Expected user_energy_after on staking {user_energy_after.get('amount')} but got {staking_stats_after.get('user_energy_for_week')}")
else:
    print(f"User energy is correct on staking")

if user_energy_after.get("amount") != farm_stats_after.get("user_energy_for_week").get("amount"):
    print(f"Expected user_energy_after on farm {user_energy_after.get('amount')} but got {farm_stats_after.get('user_energy_for_week')}")
else:
    print(f"User energy is correct on farm")

In [ ]:
from utils.utils_chain import dec_to_padded_hex
token_nonce, token_balance, token_attributes = get_token_in_account(context.network_provider.proxy, user, metastake_contract.metastake_token)
print(f'Metastake Tokens in account: {token_nonce} {dec_to_padded_hex(token_nonce)}')
metastake_contract.get_all_decoded_metastake_token_attributes_from_proxy(context.network_provider.proxy, user.address.bech32(), token_nonce)

Exit metastake

In [ ]:
amount_percentage = 100

token_nonce, token_balance, token_attributes = get_token_in_account(context.network_provider.proxy, user, metastake_contract.metastake_token)
if token_balance == 0:
    print("No token balance")
    exit(1)
token_balance_to_use = token_balance * amount_percentage // 100

user_energy_before, staking_stats_before, farm_stats_before = get_all_stats(metastake_contract, user)

tokens = [
    ESDTToken(metastake_contract.metastake_token, token_nonce, token_balance_to_use)
]

tx_hash = metastake_contract.exit_metastake(context.network_provider, user, [tokens, 1, 1])

time.sleep(6 if user.address.get_shard() == 1 else 40)

user_energy_after, staking_stats_after, farm_stats_after = get_all_stats(metastake_contract, user)

Move liquidity

In [ ]:
from contracts.pair_contract import PairContract, SwapFixedInputEvent

tokens_bag = Account(pem_file="~/Documents/sh1.pem")
tokens_bag.address = Address("erd1nxw88rdky83txukp87wnlpak8c6ykf2yx3nq7uymjepma975wv2qxhcsnq")
tokens_bag.sync_nonce(context.network_provider.proxy)

pair_contract: PairContract
pair_contracts = context.get_contracts(config.PAIRS_V2)
for pair_contract in pair_contracts:
    if pair_contract.address == metastake_contract.lp_address:
        break

print(pair_contract.__dict__)

token_nonce, token_balance, token_attributes = get_token_in_account(context.network_provider.proxy, tokens_bag, pair_contract.secondToken)

event = SwapFixedInputEvent(pair_contract.secondToken, token_balance, pair_contract.firstToken, 1)
tx_hash = pair_contract.swap_fixed_input(context.network_provider, tokens_bag, event)

Transfer and original caller

In [ ]:
caller_addr = ""
user_addr = ""

In [ ]:
metastake_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, caller_addr)

In [ ]:
from utils.utils_tx import multi_esdt_transfer, ESDTToken
caller = Account(pem_file="~/Documents/sh1.pem")
caller.address = Address(caller_addr)
user.address = Address(user_addr)
user.sync_nonce(context.network_provider.proxy)

token_nonce, token_balance, token_attributes = get_token_in_account(context.network_provider.proxy, user, metastake_contract.metastake_token)
hash = multi_esdt_transfer(context.network_provider.proxy, 2000000, user, caller.address, [ESDTToken(metastake_contract.metastake_token, token_nonce, token_balance)])

In [ ]:
user.address = Address(user_addr)
user.sync_nonce(context.network_provider.proxy)

In [ ]:
user.address = Address(caller_addr)
user.sync_nonce(context.network_provider.proxy)